# VOICEBOXAPIを呼び出すためのコード

In [ ]:
import json
import requests
import simpleaudio as sa
import io
import wave

def generate_and_play_wav(text, speaker=1):
    host = 'localhost'
    port = 50021
    params = (
        ('text', text),
        ('speaker', speaker),
    )

    # 音声合成のためのクエリを送信
    response1 = requests.post(
        f'http://{host}:{port}/audio_query',
        params=params
    )

    headers = {'Content-Type': 'application/json',}

    # 合成された音声データを取得
    response2 = requests.post(
        f'http://{host}:{port}/synthesis',
        headers=headers,
        params=params,
        data=json.dumps(response1.json())
    )

    # バイナリデータをwaveオブジェクトとして読み込む
    with io.BytesIO(response2.content) as audio_io:
        with wave.open(audio_io, 'rb') as wave_read:
            # simpleaudioで再生可能なオーディオオブジェクトを生成
            audio_data = wave_read.readframes(wave_read.getnframes())
            wave_obj = sa.WaveObject(audio_data, wave_read.getnchannels(), wave_read.getsampwidth(), wave_read.getframerate())

    # WAVファイル再生部分
    play_obj = wave_obj.play()
    play_obj.wait_done()  # 再生が完了するまで待つ

# MeMGPTを呼び出す

In [ ]:
import requests

#url = "http://localhost:8283/api/agents/9c8c4ec9-ab5c-4401-98ca-3cbdf1a98420/messages"
url = "http://localhost:8283/api/agents/<agent_id>/messages"

payload = {
    "name": "aaaaaa",
    "message": "自己紹して"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer ilovellms"
}

response = requests.post(url, json=payload, headers=headers)



In [ ]:
response

<Response [200]>

In [ ]:
import json

# response.text から得たJSON文字列

# JSONデータを解析
data = json.loads(response.text)

# 'assistant_message' の値を取得
for message in data['messages']:
    if 'assistant_message' in message:
        print(message['assistant_message'])
        generate_and_play_wav(message['assistant_message'])

再度我が名を呼びたいか？ならば、改めて紹介しよう。我はギルガメッシュ、英雄王である。人間と神の間の子で、他者がたどることのできない道を開いた英雄。ウルクの統治者で、文明の繁栄を見続けた者だ。我が存在が人間と世界の架け橋となる。忘れるな、貴様。


# MemGPTを対話型で呼び出す

In [ ]:
import requests
import json

# APIエンドポイント
#url = "http://localhost:8283/api/agents/9c8c4ec9-ab5c-4401-98ca-3cbdf1a98420/messages"
url = "http://localhost:8283/api/agents/<agent_id>/messages"
# ヘッダー情報
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer ilovellms"
}

# ユーザー入力をループで処理
while True:
    user_input = input("メッセージを入力してください（終了するには '終了' と入力）: ")
    
    # 終了条件のチェック
    if user_input.lower() == "終了":
        print("プログラムを終了します。")
        break

    # 送信するペイロード
    payload = {
        "name": "aaaaaa",
        "message": user_input
    }

    # APIへのPOSTリクエスト
    response = requests.post(url, json=payload, headers=headers)

    # レスポンスがJSON形式と仮定して解析
    if response.status_code == 200:
        data = response.json()
        # 'assistant_message' の値を取得して出力
        for message in data['messages']:
            if 'assistant_message' in message:
                print("アシスタントのメッセージ:", message['assistant_message'])
                # 音声合成と再生の関数を呼び出す（関数の実装が必要）
                generate_and_play_wav(message['assistant_message'])
    else:
        print("エラー:", response.status_code, response.text)


アシスタントのメッセージ: うむ、理解したというのなら良い。だが、我の名を理解したら何だ。貴様もそれに見合った成果を示さねばならぬ。失礼を許さぬのだ、忘れるなよ。
アシスタントのメッセージ: 私が男か女かと問いたいのか? 面白い質問だな。我は英雄王、ギルガメッシュだ。古代のメソポタミアを治めた統治者であり、人間と神の間の存在だ。男と見做されることが多いが、我を単純な性別で定義することはできぬ。我は我であることが重要だ。それが理解できるか、太田よ？
プログラムを終了します。
